<a href="https://colab.research.google.com/github/greyhound101/work_in_progress/blob/master/multifit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')
drive.mount("/content/gdrive", force_remount=True)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
Mounted at /content/gdrive


In [0]:
pip install git+https://github.com/n-waves/multifit

  Cloning https://github.com/n-waves/multifit to /tmp/pip-req-build-z76fftcu
  Running command git clone -q https://github.com/n-waves/multifit /tmp/pip-req-build-z76fftcu
  Created wheel for multifit: filename=multifit-1.0-cp36-none-any.whl size=24506 sha256=2bf03f5e673f047917694b66f2608227a06eefd443c38faededf47a3a11082ff
  Stored in directory: /tmp/pip-ephem-wheel-cache-tjlihycu/wheels/b7/92/1e/246f31a4e84fd665b5907cb96765f696be45e814ff68a5fd4a
Successfully built multifit


In [0]:
pip install sacremoses

In [0]:
pip install sentencepiece

In [0]:
from zipfile import ZipFile 
path = F"/content/gdrive/My Drive/multilingual toxic/" 
with ZipFile(path+'toxic.zip', 'r') as zip: 
    # printing all the contents of the zip file 
    zip.printdir() 
  
    # extracting all the files 
    print('Extracting all the files now...') 
    zip.extractall() 
    print('Done!')
import pandas as pd
def load_data():
    trn=pd.read_csv('jigsaw-toxic-comment-train.csv',usecols=['toxic','comment_text'])
    val=pd.read_csv('validation.csv',usecols=['toxic','comment_text','lang'])
    tst=pd.read_csv('test.csv',usecols=['lang','content'])    
    return trn,tst,val
train,test,valid=load_data()

File Name                                             Modified             Size
jigsaw-toxic-comment-train.csv                 2020-06-08 23:38:40     95538001
test.csv                                       2020-06-08 23:39:02     28783206
validation.csv                                 2020-06-08 23:38:52      3178555
submission (1).csv                             2020-06-08 23:50:22      1161879
Extracting all the files now...
Done!


In [0]:
pip install transformers

In [0]:
import os

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
import transformers
from transformers import TFAutoModel, AutoTokenizer
from tqdm.notebook import tqdm
from tokenizers import Tokenizer, models, pre_tokenizers, decoders, processors
from tensorflow.keras.layers import *
def regular_encode(texts, tokenizer, maxlen=512):
    enc_di = tokenizer.batch_encode_plus(
        texts, 
        return_attention_masks=False, 
        return_token_type_ids=False,
        pad_to_max_length=True,
        max_length=maxlen
    )
    
    return np.array(enc_di['input_ids'])
from fastai.text import *
import multifit
exp = multifit.from_pretrained("es_multifit_paper_version")
fa_config =  exp.pretrain_lm.tokenizer.get_fastai_config(add_open_file_processor=True)

In [0]:
sub=pd.read_csv('submission (1).csv')
test['toxic']=sub['toxic']
es=test.loc[test['lang']=='es']
es['id']=es.index
es=es.reset_index(drop=True)
es['toxic']=es['toxic'].round().astype('long')
es.to_csv('train.csv')
es.to_csv('test.csv')
es.to_csv('cls.csv')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [0]:
load_data(cls_dataset.cache_path/'cls')

FileNotFoundError: ignored

In [0]:
exp = multifit.from_pretrained("es_multifit_paper_version")
cls_dataset = exp.arch.dataset(Path(''), exp.pretrain_lm.tokenizer)
cls_dataset.load_clas_databunch(bs=exp.finetune_lm.bs).show_batch()


Copy sp model from /root/.fastai/models/es_multifit_paper_version to models/sp15k
Data lm-notst, trn: 8439, val: 8439
Size of vocabulary: 15000
First 20 words in vocab: ['xxunk', 'xxpad', 'xxbos', 'xxfld', 'xxmaj', 'xxup', 'xxrep', 'xxwrep', '<unk>', '▁', '▁de', '▁,', '▁.', '▁la', 's', '▁el', '▁en', '▁y', '▁a', "▁&'"]
Running tokenization: 'cls' ...


TypeError: ignored

Running tokenization: 'lm-notst' ...


Data lm-notst, trn: 8439, val: 8439
Size of vocabulary: 15000
First 20 words in vocab: ['xxunk', 'xxpad', 'xxbos', 'xxfld', 'xxmaj', 'xxup', 'xxrep', 'xxwrep', '<unk>', '▁', '▁de', '▁,', '▁.', '▁la', 's', '▁el', '▁en', '▁y', '▁a', "▁&'"]
Running tokenization: 'cls' ...


TypeError: ignored

In [0]:
x=pd.Series([1,2,3,4,'5'])
list(OrderedDict.fromkeys(x).keys()).sort()

TypeError: ignored

In [0]:
    exp.classifier.base =learn
    exp.classifier.base.encoder_fname='enc'
    data_clas = TextClasDataBunch.from_df("", es,es[:20],  text_cols = 'content', label_cols = 'toxic',vocab=data_lm.vocab)
    learn=exp.classifier.get_learner(data_clas)


Using Label smoothing with eps =  0.05
Setting Classifier weights seed seed to 0
Training args:  {'drop_mult': 0.5, 'wd': 0.01, 'pretrained': False, 'bptt': 70, 'loss_func': FlattenedLoss of LabelSmoothingCrossEntropy(), 'clip': 0.12, 'silent': False} config:  {'emb_sz': 400, 'n_hid': 1550, 'n_layers': 4, 'pad_token': 1, 'qrnn': True, 'bidir': False, 'output_p': 0.25, 'hidden_p': 0.1, 'input_p': 0.2, 'embed_p': 0.02, 'weight_p': 0.15}
Loading pretrained model enc
Setting Classifier training seed seed to 0


In [0]:
len(data_lm.vocab.itos)

15000

In [0]:
len(data_clas.vocab.itos)

15000

In [0]:
es

,content,lang,toxic,id
0,el skate es unos de los deportes favoritos de ...,es,0,7
1,Me doy la bienvenida. A este usuari le gusta c...,es,0,8
2,"ES NOTABLEMENTE TENDENCIOSO, NO SE HABLA DE CU...",es,0,9
3,"El Jardín de infantes Nº938, fundado en 1989, ...",es,0,22
4,Daré explicaciones y/o aclaraciones a cualquie...,es,0,49
...,...,...,...,...
8433,Los mongoles emplearon una crueldad apabullant...,es,0,63783
8434,I N S T I - TI! T U - TU! T O - TO! INSTITUTO ...,es,0,63787
8435,Te agradezco tu cordial mensaje. De los cambio...,es,0,63804
8436,"Amigo mio, quien diría que esta travesía de lo...",es,0,63805


In [0]:
data_lm.show_batch()

idx,text
0,"xxup ▁no ▁ xxup ▁se ▁ xxup ▁habla ▁ xxup ▁de ▁ xxup ▁cuando ▁ xxup ▁la ▁ xxup ▁tu pac ▁ xxup ▁amenaza ba ▁y ▁ xxup ▁fue ▁ xxup ▁denuncia do ▁ xxup ▁por ▁3 ▁ xxup ▁fiscal es ▁ xxup ▁en ▁ xxup ▁febrero ▁, ▁ xxup ▁ni ▁ xxup ▁de ▁ xxup ▁cama ras ▁ xxup ▁oculta s ▁, ▁ xxup ▁ni ▁ xxup ▁de ▁"
1,"▁que ▁ te ▁importa ▁no ▁tiene s ▁mejores ▁cosas ▁que ▁hacer ▁ ? ▁fa ck ú ▁( ▁ xxup ▁ mb c ▁hip - hop ▁crea tion ▁) ▁ xxbos ▁ xxfld ▁1 ▁ xxmaj ▁pues ▁me ▁per do na rá s ▁pero ▁si ▁ ve o ▁algún ▁ pla gio ▁en ▁el ▁que ▁solo ▁haya ▁con tribu ido ▁él ▁o ▁haya ▁ediciones ▁menores ▁de ▁otros ▁, ▁lo ▁pasa ré"
2,▁normas ▁de ▁un ▁idioma . si ▁no ▁se ▁hace ▁así ▁a i ▁can ▁esp ik ▁en gli s ▁tu ▁ bat ▁in ▁mai ▁o un ▁ ue i ▁. ▁ xxmaj ▁bu fa lo ▁ xxbos ▁ xxfld ▁1 ▁cata lan es ▁hijos ▁de ▁pu ta . ▁cata lan es ▁hijos ▁de ▁pu ta . ▁cata lan es ▁hijos ▁de ▁pu ta . ▁cata lan es ▁hijos ▁de ▁pu ta
3,"xxup ▁fueron ▁ xxup ▁los ▁ xxup ▁mont on eros ▁. ▁ xxbos ▁ xxfld ▁1 ▁ xxup ▁ edi t ▁: ▁ xxmaj ▁philip ▁ xxup ▁ k . ▁ xxmaj ▁di ck ▁ is ▁ ali ve ▁and ▁we ▁a re ▁all ▁de ad ▁ xxmaj ▁a y ▁, ▁ xxmaj ▁ ubi k ▁, ▁qué ▁excelente ▁película ▁ha ría ▁con ▁ vo s ▁un ▁director ▁inteligente ▁..."
4,"▁en ▁artículo ▁. ▁ xxmaj ▁a ten ta mente ▁, ▁ xxmaj ▁daniel a ▁ xxbos ▁ xxfld ▁1 ▁ xxmaj ▁esp ero ▁paciente mente ▁una ▁respuesta ▁ - ▁) ▁ xxmaj ▁los ▁biblioteca rios ▁re vi sa mos ▁cuidado sa mente ▁cualquier ▁marcado ▁para ▁borra r ▁antes ▁de ▁eliminar lo ▁, ▁pero ▁eso ▁no ▁ quita ▁que ▁haya ▁que ▁marca rlo s ▁con ▁cuidado ▁. ▁ xxmaj ▁sólo ▁"
